In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
import pathlib
from skimage.measure import regionprops, label
from pathlib import Path
import torch
import cv2
import edt
import math
import matplotlib as mpl
from math import floor
%matplotlib qt

In [2]:
filepath = Path('/home/pk/Documents/rtseg/data/backbone/000_img_masks.tif')

In [3]:
image = imread(filepath)

In [4]:
plt.figure()
plt.imshow(image)
plt.show()

In [5]:
np.unique(image)

array([  0, 448, 462, 489, 503, 510, 513, 528, 548, 554, 557, 570, 579,
       590, 603, 619, 631], dtype=uint16)

#### __Compute the backbone of each region on the image__

#### Fixed parameters

In [6]:
props = regionprops(image)

In [7]:
props[0].bbox, props[0].label

((10, 16, 41, 36), 448)

In [8]:
plt.figure()
plt.imshow(props[0].image)
plt.show()

In [9]:
dilation_threshold = 3
dilation_radius = floor(dilation_threshold/2)

In [10]:
dilation_radius

1

In [11]:
img = props[0].image

In [12]:
padded_img = np.pad(img, (dilation_radius+1, dilation_radius+1)).astype('uint8')

In [13]:
img.shape, padded_img.shape

((31, 20), (35, 24))

In [14]:
se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (dilation_radius+2, dilation_radius+2))

In [15]:
se

array([[0, 1, 0],
       [1, 1, 1],
       [0, 1, 0]], dtype=uint8)

In [16]:
dilated_img = cv2.dilate(padded_img, se)

In [17]:
dilated_img.shape

(35, 24)

In [18]:
xor = np.logical_xor(padded_img, dilated_img)

In [19]:
plt.imshow(xor)

In [20]:
plt.imshow(dilated_img)

In [21]:
region_edt = edt.edt(dilated_img)

In [22]:
plt.imshow(region_edt)

In [23]:
blob_boundary = (region_edt == 1)

In [24]:
label_dist = region_edt * (region_edt >= dilation_threshold)

In [25]:
final_labelled_image = label_dist[dilation_radius+1: -dilation_radius-1, dilation_radius+1: -dilation_radius-1]

In [26]:
final_labelled_image.shape

(31, 20)

In [27]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(props[0].image)
ax[1].imshow(final_labelled_image)
plt.show()

In [28]:
y_coords, x_coords = np.nonzero(final_labelled_image)

In [29]:
pixel_values = final_labelled_image[y_coords, x_coords]

In [30]:
y_coords.shape, x_coords.shape, pixel_values.shape

((190,), (190,), (190,))

#### I am changing this cubing to a parameter, as I notice some of them are not great

In [31]:
weights = np.sqrt(pixel_values ** 3)

In [32]:
weights.shape

(190,)

In [33]:
A = np.vstack((x_coords**2, x_coords, np.ones(x_coords.shape)))

In [34]:
A.shape

(3, 190)

In [35]:
w_rep = np.tile(weights, [3, 1])

In [36]:
wA = (w_rep * A).T
wB = weights * y_coords

In [37]:
wA.shape, wB.shape

((190, 3), (190,))

In [38]:
from numpy.linalg import lstsq

In [39]:
fit_coeff,_, _, _ = lstsq(wA, wB, rcond=-1)

In [40]:
fit_coeff

array([ 0.01549774,  1.60419873, -3.85620333])

In [41]:
plt.figure()
plt.imshow(final_labelled_image)
x = np.linspace(0.5, final_labelled_image.shape[1]-0.5, num=100)
y = fit_coeff[0] * x**2 + fit_coeff[1] * x + fit_coeff[2]
plt.plot(x, y, 'r-')
plt.show()

#### Compute backbone pixels

In [42]:
img.shape

(31, 20)

In [43]:
img_size = img.shape

In [44]:
x_data = np.arange(-0.5, img_size[1]+0.5)
y_data = fit_coeff[0] * x_data**2 + fit_coeff[1] * x_data + fit_coeff[2]

In [45]:
plt.figure()
plt.imshow(img)
plt.plot(x_data, y_data, 'r--')
plt.show()

##### There are cases. one for a straight line and the other for a quadratic curve

In [48]:
if abs(fit_coeff[0]) < 1e-4 and round(y_data[0]) == round(y_data[-1]):
    fit_coord = np.vstack((np.arange(0, img_size[1]), round(y_data[0]) * np.ones((img_size[1],)))).T
else:
    y_data_round = np.round(y_data)
    stat_pt = -fit_coeff[1] / (2 * fit_coeff[0]) #  ax^2 + bx + c = 0 --> -b/2a is the point where derivative==0
    if stat_pt > -0.5 and stat_pt < img_size[1] - 1:
        round_stat_pt_val = round(fit_coeff[0] * (stat_pt**2) + fit_coeff[1] * stat_pt + fit_coeff[2])
        round_stat_pt = round(stat_pt)
        upp_bound = np.zeros((img_size[1]),) # upper bound defined for each pixel on the x-axis
        low_bound = np.zeros((img_size[1]),) # lower bound defined for each pixel on the x-axis
    
        if fit_coeff[0] > 0:
            upp_bound[0:round_stat_pt] = y_data_round[0:round_stat_pt]
            upp_bound[round_stat_pt] = max(y_data_round[round_stat_pt:round_stat_pt+2])
            upp_bound[round_stat_pt+1:] = y_data_round[round_stat_pt+1:]
    
            low_bound[0:round_stat_pt] = y_data_round[1:round_stat_pt+1]
            round_down_point = np.argwhere(y_data_round[1:] - y_data[1:] == 0.5)
            low_bound[round_down_point] = lowBound[round_down_point]-1 
            low_bound[round_stat_pt] = round_stat_pt_val 
            low_bound[round_stat_pt+1:] =  y_data_round[round_stat_pt+1:]
                                           
        else:
            upp_bound[0:round_stat_pt] = y_data_round[1:round_stat_pt+1]
            upp_bound[round_stat_pt] = round_stat_pt_val
            upp_bound[round_stat_pt+1:] = y_data_round[round_stat_pt:-1];
            low_bound[0:round_stat_pt] =  y_data_round[0:round_stat_pt]
            low_bound[round_stat_pt] = max(y_data_round[round_stat_pt:round_stat_pt+2])
            low_bound[round_stat_pt+1:] = y_data_round[round_stat_pt+1:]
    else:
        if (fit_coeff[0] > 0 and stat_pt > img_size[1]-1) or (fit_coeff[0] < 0 and stat_pt < -0.5):
            upp_bound = y_data_round[:-1]
            low_bound = y_data_round[1:]
            round_down_point = np.argwhere(low_bound - y_data[1:] == 0.5)
            low_bound[round_down_point] = low_bound[round_down_point]-1;
        else:
            upp_bound = y_data_round[1:]
            low_bound = y_data_round[:-1]
        
    
    # clip the upper bound and lower bounds
    # and populate the pixels
    low_bound = np.maximum(low_bound, 0).astype('int')
    upp_bound = np.minimum(upp_bound, img_size[0]-1).astype('int')
    n_pixels_per_col = upp_bound - low_bound + 1
    n_pixels_per_col[n_pixels_per_col < 1] = 0
    fit_coord = np.zeros((int(np.sum(n_pixels_per_col)), 2))
    idx = 0;
    for i in range(len(n_pixels_per_col)):
        fit_coord[idx:idx+n_pixels_per_col[i], 0] = i;
        fit_coord[idx:idx+n_pixels_per_col[i], 1] = np.arange(low_bound[i], upp_bound[i]+1)
        idx += n_pixels_per_col[i]
    
    fit_coord = fit_coord.astype('int')

### Poles

In [49]:
fit_im = np.zeros(img_size, dtype='bool')
fit_im[fit_coord[:, 1], fit_coord[:, 0]] = True
backbone_im = img * fit_im

y_backbone, x_backbone = np.nonzero(backbone_im)

In [50]:
poles = np.zeros((2, 2))

In [51]:
poles[:, 0] = [x_backbone[0] - 0.5, x_backbone[-1] + 0.5]
poles[:, 1] = fit_coeff[0] * poles[:, 0]**2 + fit_coeff[1] * poles[:, 0] + fit_coeff[2]

In [52]:
poles

array([[ 1.5       , -1.41503532],
       [17.5       , 28.96345827]])

In [54]:
plt.figure()
plt.imshow(img)
plt.plot(poles[:, 0], poles[:, 1], '*')
plt.show()

### Arc length

In [55]:
x_0, x_end = x_backbone[0] - 0.5, x_backbone[-1] + 0.5

if abs(fit_coeff[0]) > 1e-10:
    transf_points = (fit_coeff[0] / abs(fit_coeff[0])) * np.array([2*fit_coeff[0]*x_0 + fit_coeff[1] , 2*fit_coeff[0]*x_end + fit_coeff[1]])
    sqrt_exp = np.sqrt(1 + transf_points**2)
    arc_length = np.diff(transf_points * sqrt_exp + np.log(transf_points + sqrt_exp)) / (4 * abs(fit_coeff[0]))
else:
    
    fit_points = [fit_coeff[1] * x_0 + fit_coeff[2], fit_coeff[1] * x_end + fit_coeff[2]]
    arc_length = math.sqrt((x_0 - x_end) ** 2 + (fit_points[0] - fit_points[1]) ** 2)
    

In [56]:
print(arc_length)

[34.35111691]



### Just do a dilation and see if a dot lands on the cell and assign the dot coordinates. Just that simple.
#### Be careful and loop over all the dots as each cell can have more than one dot.

### Compute internal coordinates

In [57]:
poles

array([[ 1.5       , -1.41503532],
       [17.5       , 28.96345827]])

In [58]:
dot_coordinate = np.array([[8.0, 15.0]])

In [60]:
plt.figure()
plt.imshow(img)
plt.plot(x_data, y_data, 'r--')
plt.plot(poles[:, 0], poles[:, 1], '*')
plt.plot(dot_coordinate[:, 0], dot_coordinate[:, 1], 'go')
plt.show()

#### Project normal and solve a cubic equation

In [68]:
dot_no = 0
p0, p1, p2 = fit_coeff
x0, y0 = dot_coordinate[dot_no]
projected_point = np.zeros(dot_coordinate[dot_no].shape)
if abs(p0) > 1e-10:
    x_star = np.roots([2*p0**2, 3*p0*p1, 2*p0*(p2-y0) + (p1**2+1), p1*(p2-y0)-x0]) # 3rd degree polynomial
    x_star[abs(np.imag(x_star)) < 1e-10] = np.real(x_star[abs(np.imag(x_star)) < 1e-10])
    #print(x_star)
    x_star = np.real(x_star[np.isreal(x_star)])
    x_mid = -p1/(2*p0)
    if x0 > x_mid:
        projected_point[0] = np.max(x_star)
    else:
        projected_point[0] = min(x_star)
else:
    projected_point[0] = x0 - ((p1 * (p2 - p1)) / ((p1**2 + 1)))

projected_point[1] = p0 * projected_point[0] ** 2 + p1 * projected_point[0] + p2
# you should return projected point and distance to projected point.

distance_to_proj = math.sqrt((projected_point[0] - x0)**2 + (projected_point[1] - y0)**2)

In [70]:
plt.figure()
plt.imshow(img)
plt.plot(x_data, y_data, 'r--')
plt.plot(poles[:, 0], poles[:, 1], '*')
plt.plot(dot_coordinate[:, 0], dot_coordinate[:, 1], 'go')
plt.plot(projected_point[0], projected_point[1], 'k*')
plt.plot([dot_coordinate[0, 0], projected_point[0]], [dot_coordinate[0, 1], projected_point[1]], 'b--')
plt.show()

In [71]:
distance_to_proj

2.3581039112649806

In [ ]:
def compute_backbone_poles():
    """
    For each cell in the image, when you call regionprops, you also want to 
    calculate the backbones and poles of each cell in the image.
    This function is meant to be in the extra_properties arguement in 
    skimage.measure.regionprops fuction.

    This function will add extra properties to each region in the 
    regionprops list.

    The extra properties are 
        1. fit_coeff: [a, b, c] 
        2. poles: 
        3. arc_length: 
        4. backbone_coordinates:

    Backbone is fit using a quadratic equation.

    
    Args:

    Returns:
    
    """
    pass
def compute_arc_length():
    """
    Computes the arc length
    """
    pass

#### __Computing internal coordinate of the dot__

In [ ]:
def get_projected_point():
    """
    
    """
    pass

In [16]:
def compute_internal_coordinate():
    """
    """
    pass